# Segmenting and Clustering Neighborhoods in Toronto

For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

#### Loading common libraries

In [10]:
import numpy as np 
import pandas as pd 

#### Loading Module BeautifulSoup

In [11]:
from bs4 import BeautifulSoup
import requests

#### Loading Wiki List of postal codes of Canada: M 

In [12]:
html = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

#### Parse HTML and create a list for HTML Table

In [13]:
soup = BeautifulSoup(html.text, 'html.parser')
tablesoup=soup.find("table")

def get_string(ele):
    if ele.find("a") != None :
        return ele.find("a").string.replace('\n','')
    else:
        return ele.string.replace('\n','')
    
my_list=[]
for row_td in tablesoup.find_all("tr"):
    row=row_td.find_all("td")
    if len(row) != 0 :
        my_list.append([get_string(row[0]),get_string(row[1]),get_string(row[2])])
#print(my_list)

#### Creating dataframe with above table

In [14]:
# This dataframe is not cleaned yet
df=pd.DataFrame(my_list,columns=['PostalCode','Borough','Neighborhood'])
df.reset_index(drop=True)
df.shape

(287, 3)

#### Removing cells Borough = "Not assigned"

In [15]:
#cleaning data
df=df[df.Borough != 'Not assigned']

df.shape

(210, 3)

#### In this step we will concatenate the Neighborhood with ',' for the same PostalCode and remove duplicate PostalCode
Using lamda function with lookup

In [16]:
def concat_str(narray):
    new_series = pd.Series(narray)
    s=new_series.str.cat(sep=',')
    return s
    
df['Neighborhood']=df.apply( lambda row: row['Neighborhood'] if ((df[df['PostalCode'] == row['PostalCode']].count())[0] < 2) else concat_str(df[df['PostalCode'] == row['PostalCode']]['Neighborhood'].unique()), axis=1)

df.drop_duplicates(subset ="PostalCode",keep = 'first',inplace=True) 

df.shape

(103, 3)

#### In this step for "Not assigned" Neighborhood we will assign the Borough value.  
I used lamda function inorder to achive it

In [17]:
df['Neighborhood']=df.apply( lambda row: row['Borough'] if (row['Neighborhood'] == 'Not assigned') else row['Neighborhood'], axis=1)

df.shape


(103, 3)

#### Displaying shape

In [18]:
df.shape

(103, 3)

#### Displaying full dataframe


In [19]:
df.reset_index(drop=True,inplace=True)
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Downtown Toronto,Queen's Park
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


#### method for location lat lng

In [4]:
!conda install -c conda-forge geocoder --yes 

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          59 KB

The following NEW packages will be INSTALLED:

    geocoder: 1.38.1-py_1 conda-forge
    ratelim:  0.1.6-py_2  conda-forge


geocoder-1.38.1      | 53 KB     | ##################################### | 100% 
ratelim-0.1.6        | 6 KB      | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [20]:
import geocoder 

####  Note: The geocoder API didn't work it was taking very long so I had to use the csv given

In [25]:
!wget -q -O 'Geospatial_data.csv' https://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [26]:
geo_df = pd.read_csv('Geospatial_data.csv')
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Copying the Latitude and Longitude columns to df

In [36]:
df['Latitude']=geo_df['Latitude']
df['Longitude']=geo_df['Longitude']

#### Two new columns added to df 

In [38]:
df.shape

(103, 5)

In [39]:
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.806686,-79.194353
1,M4A,North York,Victoria Village,43.784535,-79.160497
2,M5A,Downtown Toronto,Harbourfront,43.763573,-79.188711
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.770992,-79.216917
4,M7A,Queen's Park,Queen's Park,43.773136,-79.239476
5,M9A,Downtown Toronto,Queen's Park,43.744734,-79.239476
6,M1B,Scarborough,"Rouge,Malvern",43.727929,-79.262029
7,M3B,North York,Don Mills North,43.711112,-79.284577
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.716316,-79.239476
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.692657,-79.264848


### Let's get the geographical coordinates of Toronto.

In [40]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [43]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.0.0               |             py_0         606 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         704 KB

The following NEW packages will be INSTALLED:

    altair:  4.0.0-py_0 conda-forge
    branca:  0.3.1-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge


altair-4.0.0         | 606 KB    | ##################################### | 1

In [44]:
import folium

visualizat Toronto the neighborhoods in it.

In [45]:
# create map of Toronto using latitude and longitude values
map_on = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_on)  
    
map_on

In [56]:
on_data = df[df['Borough'].str.find('Toronto') != -1].reset_index(drop=True)
on_data.shape

(39, 5)

In [110]:
# The code was removed by Watson Studio for sharing.

#### Explore the first neighborhood in our dataframe.

In [57]:
on_data.loc[0, 'Neighborhood']

'Harbourfront'

Get the neighborhood's latitude and longitude values.

In [58]:
neighborhood_latitude = on_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = on_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = on_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Harbourfront are 43.7635726, -79.1887115.


In [59]:
# type your answer here
url="https://api.foursquare.com/v2/venues/search?client_id="+str(CLIENT_ID)+"&client_secret="+str(CLIENT_SECRET)+"&ll="+str(neighborhood_latitude)+","+str(neighborhood_longitude)+"&radius=500&limit=100&v="+VERSION


In [60]:
results = requests.get(url).json()

In [61]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [63]:
from pandas.io.json import json_normalize

In [66]:
venues = results['response']['venues']
    
nearby_venues = json_normalize(venues) # flatten JSON

# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in nearby_venues.columns if col.startswith('location.')] + ['id']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
nearby_venues['categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean column names by keeping only last term
nearby_venues.columns = [column.split('.')[-1] for column in nearby_venues.columns]

nearby_venues.shape


(67, 15)

In [67]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

67 venues were returned by Foursquare.


In [70]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            '100')
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [71]:
# type your answer here

on_venues = getNearbyVenues(names=on_data['Neighborhood'],
                                   latitudes=on_data['Latitude'],
                                   longitudes=on_data['Longitude']
                                  )



In [74]:
print(on_venues.shape)

(774, 7)


In [77]:
on_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",3,3,3,3,3,3
Berczy Park,1,1,1,1,1,1
"Brockton,Exhibition Place,Parkdale Village",41,41,41,41,41,41
Business Reply Mail Processing Centre 969 Eastern,4,4,4,4,4,4
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",19,19,19,19,19,19
"Cabbagetown,St. James Town",3,3,3,3,3,3
Central Bay Street,5,5,5,5,5,5
"Chinatown,Grange Park,Kensington Market",37,37,37,37,37,37
Christie,2,2,2,2,2,2


In [78]:
print('There are {} uniques categories.'.format(len(on_venues['Venue Category'].unique())))

There are 206 uniques categories.


## Analyze Each Neighborhood

In [81]:
# one hot encoding
on_onehot = pd.get_dummies(on_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
on_onehot['Neighborhood'] = on_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [on_onehot.columns[-1]] + list(on_onehot.columns[:-1])
on_onehot = on_onehot[fixed_columns]

on_onehot.shape

(774, 206)

In [82]:
on_onehot.shape

(774, 206)

In [84]:
on_grouped = on_onehot.groupby('Neighborhood').mean().reset_index()

In [85]:
on_grouped.shape

(39, 206)

In [86]:
num_top_venues = 5

for hood in on_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = on_grouped[on_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                      venue  freq
0                   Airport  0.33
1                      Park  0.33
2               Snack Place  0.33
3                 Nightclub  0.00
4  Mediterranean Restaurant  0.00


----Berczy Park----
                venue  freq
0           Cafeteria   1.0
1     Organic Grocery   0.0
2  Light Rail Station   0.0
3      Lingerie Store   0.0
4        Liquor Store   0.0


----Brockton,Exhibition Place,Parkdale Village----
                 venue  freq
0                 Café  0.10
1          Coffee Shop  0.07
2  American Restaurant  0.05
3            Gastropub  0.05
4   Italian Restaurant  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0   Mobile Phone Shop  0.25
1                Park  0.25
2         Pizza Place  0.25
3            Bus Line  0.25
4  Miscellaneous Shop  0.00


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
         

In [87]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [90]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = on_grouped['Neighborhood']

for ind in np.arange(on_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(on_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.shape

(39, 11)

In [92]:
from sklearn.cluster import KMeans

## Cluster Neighborhoods

Runing for 5 clusters

In [98]:
# set number of clusters
kclusters = 5

on_grouped_clustering = on_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(on_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 4, 2, 1, 2, 2, 2, 2, 1, 2], dtype=int32)

In [101]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

on_merged = on_data
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
on_merged = on_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

on_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,43.763573,-79.188711,2,Medical Center,Electronics Store,Intersection,Breakfast Spot,Pizza Place,Rental Car Location,Mexican Restaurant,Dance Studio,Dog Run,Discount Store
1,M9A,Downtown Toronto,Queen's Park,43.744734,-79.239476,2,Spa,Playground,Women's Store,Dance Studio,Donut Shop,Doner Restaurant,Dog Run,Discount Store,Diner,Dim Sum Restaurant
2,M5B,Downtown Toronto,"Ryerson,Garden District",43.692657,-79.264848,2,General Entertainment,Café,College Stadium,Skating Rink,Construction & Landscaping,Convenience Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
3,M5C,Downtown Toronto,St. James Town,43.799525,-79.318389,2,Chinese Restaurant,Fast Food Restaurant,Pharmacy,Coffee Shop,Breakfast Spot,Furniture / Home Store,Grocery Store,Sandwich Place,Burger Joint,Pizza Place
4,M4E,East Toronto,The Beaches,43.786947,-79.385975,2,Japanese Restaurant,Chinese Restaurant,Bank,Café,Department Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


In [103]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [104]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(on_merged['Latitude'], on_merged['Longitude'], on_merged['Neighborhood'], on_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

### Cluster 1

In [105]:
on_merged.loc[on_merged['Cluster Labels'] == 0, on_merged.columns[[1] + list(range(5, on_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,West Toronto,0,Pub,Trail,Health Food Store,Asian Restaurant,Women's Store,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store


### Cluster 2

In [106]:
on_merged.loc[on_merged['Cluster Labels'] == 1, on_merged.columns[[1] + list(range(5, on_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Downtown Toronto,1,Food & Drink Shop,Park,Women's Store,Donut Shop,Doner Restaurant,Dog Run,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop
8,Downtown Toronto,1,Airport,Snack Place,Park,Women's Store,Dance Studio,Donut Shop,Doner Restaurant,Dog Run,Discount Store,Diner
9,West Toronto,1,Shopping Mall,Park,Grocery Store,Bank,Hotel,Deli / Bodega,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
24,Central Toronto,1,Park,Women's Store,Fast Food Restaurant,Market,Dance Studio,Doner Restaurant,Dog Run,Discount Store,Diner,Dim Sum Restaurant
26,Central Toronto,1,Construction & Landscaping,Park,Basketball Court,Bakery,Women's Store,Department Store,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
38,East Toronto,1,Bus Line,Park,Pizza Place,Mobile Phone Shop,Women's Store,Doner Restaurant,Dog Run,Discount Store,Diner,Dim Sum Restaurant


### Cluster 3

In [107]:
on_merged.loc[on_merged['Cluster Labels'] == 2, on_merged.columns[[1] + list(range(5, on_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,2,Medical Center,Electronics Store,Intersection,Breakfast Spot,Pizza Place,Rental Car Location,Mexican Restaurant,Dance Studio,Dog Run,Discount Store
1,Downtown Toronto,2,Spa,Playground,Women's Store,Dance Studio,Donut Shop,Doner Restaurant,Dog Run,Discount Store,Diner,Dim Sum Restaurant
2,Downtown Toronto,2,General Entertainment,Café,College Stadium,Skating Rink,Construction & Landscaping,Convenience Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
3,Downtown Toronto,2,Chinese Restaurant,Fast Food Restaurant,Pharmacy,Coffee Shop,Breakfast Spot,Furniture / Home Store,Grocery Store,Sandwich Place,Burger Joint,Pizza Place
4,East Toronto,2,Japanese Restaurant,Chinese Restaurant,Bank,Café,Department Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
6,Downtown Toronto,2,Coffee Shop,Discount Store,Grocery Store,Pharmacy,Pizza Place,Curling Ice,Donut Shop,Doner Restaurant,Dog Run,Diner
10,Downtown Toronto,2,Beer Store,Cosmetics Shop,Curling Ice,Athletics & Sports,Pharmacy,Spa,Park,Video Store,Skating Rink,Dance Studio
12,East Toronto,2,Greek Restaurant,Coffee Shop,Ice Cream Shop,Furniture / Home Store,Italian Restaurant,Restaurant,Bookstore,Indian Restaurant,Juice Bar,Diner
13,Downtown Toronto,2,Park,Movie Theater,Gym,Steakhouse,Fast Food Restaurant,Sushi Restaurant,Sandwich Place,Fish & Chips Shop,Pet Store,Burrito Place
14,West Toronto,2,Café,Coffee Shop,Gastropub,Italian Restaurant,American Restaurant,Bakery,Coworking Space,Diner,Bar,Seafood Restaurant


### Cluster 4

In [108]:
on_merged.loc[on_merged['Cluster Labels'] == 3, on_merged.columns[[1] + list(range(5, on_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,Downtown Toronto,3,Construction & Landscaping,Baseball Field,Women's Store,Department Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store
36,Downtown Toronto,3,Baseball Field,Women's Store,Empanada Restaurant,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store,Diner


### Cluster 5

In [109]:
on_merged.loc[on_merged['Cluster Labels'] == 4, on_merged.columns[[1] + list(range(5, on_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Downtown Toronto,4,Cafeteria,Women's Store,Dance Studio,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store,Diner,Dim Sum Restaurant
